In [216]:
import geopandas as gpd
import numpy as np
from shapely.geometry import Point
import pandas as pd

In [68]:
gdf_nbl = gpd.read_file('zip://downloads/SIRGAS_SHP_logradouronbl.zip!SIRGAS_SHP_logradouronbl')
gdf_classificacao_viaria = gpd.read_file('zip://downloads/SIRGAS_SHP_classeviariacet.zip!SIRGAS_SHP_classeviariacet')
gdf_distritos = gpd.read_file('downloads/SIRGAS_GPKG_distrito.gpkg')
gdf_quadras = gpd.read_file('downloads/SIRGAS_SHP_quadraMDSF.zip!SIRGAS_SHP_quadraMDSF')
gdf_cruzamentos = gpd.read_file('../poligono-de-vias/resultado/pre-processamentos/00_cruzamentos_cidade_sao_paulo.gpkg')
# gdf_poligono_de_vias = gpd.read_file('../poligono-de-vias/resultado/90_poligono_de_vias_de_vila_mariana.gpkg')

In [203]:
buffer_cruzamento = 1.5
distancia_ate_cruzamento = 8.

In [200]:

for i, dist in gdf_distritos[gdf_distritos.ds_codigo == '90'].iterrows():

    print(dist.ds_nome)

    nbl = gdf_nbl.clip(dist.geometry.buffer(20))
    qvia = gdf_quadras.clip(dist.geometry.buffer(20))
    cruza = gdf_cruzamentos.clip(dist.geometry.buffer(20))

    # Exclui os cruzamentos que estão dentro das quadras viárias
    cruza = cruza[~cruza.intersects(qvia.unary_union)]

    # exclui linhas da NBL que intersectam quadas viárias
    nbl = nbl[~nbl.intersects(qvia.unary_union)]

    # gera um buffer para cada cruzamento
    cruza.geometry = cruza.buffer(1.5)
    
    # relacionar cruzamentos com NBL e manter somente as relações > 2
    cruza_nbl_group = cruza_nbl.reset_index()[['lg_seg_id', 'index']].groupby('index').agg('count')
    cruza = cruza[cruza_nbl_group['lg_seg_id'] > 2]

    # Verificar se ponto final e inicial são cruzamentos
    nbl['ponto_inicial'] = [Point(g.coords[:][0]) for g in nbl.geometry]
    nbl['ponto_final'] = [Point(g.coords[:][-1]) for g in nbl.geometry]
    nbl['faixa_ponto_inicial'] = nbl.set_geometry('ponto_inicial').intersects(cruza.unary_union)
    nbl['faixa_ponto_final'] = nbl.set_geometry('ponto_final').intersects(cruza.unary_union)

    # Gerar um ponto de faixa afastado 'distancia_ate_cruzamento' metros das extremidades que se cruzam
    pontos = []
    pontos.append(nbl[(nbl.length > 3 * distancia_ate_cruzamento) & (nbl.faixa_ponto_inicial)].geometry.apply(lambda x: x.interpolate(distancia_ate_cruzamento)))
    pontos.append(nbl[(nbl.length > 3 * distancia_ate_cruzamento) & (nbl.faixa_ponto_inicial)].geometry.apply(lambda x: x.interpolate(-distancia_ate_cruzamento)))
    gdf_faixas_ideiais = gpd.GeoDataFrame(geometry=pd.concat(pontos))

    ## Alguns problemas derivados do polígono de vias estar em uma versão anterior à atualização de quadras

    ## TODO
    # Remover as faixas de VTR




VILA MARIANA


/home/feromes/miniconda3/envs/geo/lib/python3.8/site-packages/geopandas/geodataframe.py:1327: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)
